In [17]:
import os, glob, traceback
import numpy as np
import pandas as pd
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub

In [22]:
TRAIN_CSV        = '/home/jovyan/Data/birdclef-2025/train.csv'
DEN_DIR          = '/home/jovyan/Features/denoised'
PERCH_TFHUB_URL  = "https://tfhub.dev/google/bird-vocalization-classifier/4"
SAMPLE_RATE      = 32000
CHUNK_SEC        = 5
CHUNK_SAMPLES    = SAMPLE_RATE * CHUNK_SEC
BATCH_SIZE       = 16
EPOCHS           = 3
LR               = 1e-4

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("PyTorch device:", DEVICE)

# Allow TF GPU memory growth (coexist with PyTorch)
gpus = tf.config.experimental.list_physical_devices('GPU')
for g in gpus:
    try: tf.config.experimental.set_memory_growth(g, True)
    except: pass

PyTorch device: cuda


In [1]:
from matplotlib import pyplot as plt
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
tf.experimental.numpy.experimental_enable_numpy_behavior()

# Load the model.
model = hub.load('https://www.kaggle.com/models/google/bird-vocalization-classifier/TensorFlow2/bird-vocalization-classifier/8')

# Input: 5 seconds of silence as mono 32 kHz waveform samples.
waveform = np.zeros(5 * 32000, dtype=np.float32)

# Run the model, check the output.
model_outputs = model.infer_tf(waveform[np.newaxis, :])

# Examine the spectrogram.
plt.imshow(model_outputs['frontend'][0].T)

# Examine various logits.
print(model_outputs['label'].shape)
print(model_outputs['genus'].shape)
print(model_outputs['family'].shape)
print(model_outputs['order'].shape)

# Examine the embeddings.
print(model_outputs['embedding'].shape)


2025-04-30 03:21:35.801478: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745983295.821087   77506 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745983295.827130   77506 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745983295.842936   77506 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745983295.842951   77506 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745983295.842953   77506 computation_placer.cc:177] computation placer alr

FailedPreconditionError: DNN library initialization failed. Look at the errors above for more details. [Op:__inference_restored_function_body_20537]

In [2]:
from chirp.inference import models

# Input: 5 seconds of silence as mono 32 kHz waveform samples.
waveform = np.zeros(5 * 32000, dtype=np.float32)

# Automatically downloads the model from Kaggle.
model = models.TaxonomyModelTF.load_version(8)
outputs = model.embed(audio)
# do something with outputs.embeddings and outputs.logits['label']


ModuleNotFoundError: No module named 'chirp'